In [2]:
import numpy as np                                          # 导入numpy库并简写为np
import pandas as pd
import itertools
from joblib import Parallel, delayed
import argparse
from tqdm import tqdm
from QAOA_CSP import CSP, _H,getBaseProb,_Rx,_Rz,_Rzz,_Rzzz,_Rzzzz,I,_H,getBaseProb
import time

In [3]:
def create_new_dict(nested_dict, func) -> dict:
    new_dict = {}
    for key, value in nested_dict.items():
        if isinstance(value, dict):
            new_dict[key] = create_new_dict(value, func)
        else:
            new_key, new_value = func(key, value)
            new_dict[new_key] = new_value
    
    return new_dict

# 将二层的key转化为我要的形式的字符串
def modify_key_value(key, value):
    operLst = ",".join(list(map(lambda x:str(eval(x[2]) - 1),key.split())))

    return operLst, value
def Qaoa3E(Problem:dict,HC,gammaLst:list,betaLst:list,p:int,s:np.ndarray) -> float:
    model = CSP(Problem,HC)
    model.updateQC(gammaLst,betaLst,p)
    return model.getExpctation(s)

# Problem转为经典表达式
def paraCSP(Problem,z1,z2,z3,z4):
    C = 0
    Problem_beta = Problem[1]
    Constant = Problem[2]
    factor = Problem[3]
    for _, Power in Problem_beta.items(): #每个次幂的
        for key, coef in Power.items(): #某个次幂中 每个项
            # 解析每个项的表达式，并乘value系数
            item = 1
            for oper in key.split(): 
                item  = item * eval("z{}".format(oper[2]))
            C  = C + item*coef 
    return factor*(C+Constant)
def GridSearch_CSP(Problem):
    sulotionsSet = itertools.product([-1,1],repeat=4)
    obj_maxLst = [0]
    para_max = []
    for z1,z2,z3,z4 in sulotionsSet:
        obj = paraCSP(Problem,z1,z2,z3,z4)
        if obj > obj_maxLst[0]:
            obj_maxLst = [obj]
            para_max = [(z1,z2,z3,z4)]
        elif obj == obj_maxLst[0]:
            obj_maxLst.append(obj)
            para_max.append((z1,z2,z3,z4))
        else:
            pass
    return obj_maxLst,para_max
# 结果评价指标 信息熵

def Entropy(probabilities:np.ndarray):
    # 计算数组中每个元素的概率
    # 计算信息熵
    entropy = -np.sum(probabilities * np.log(probabilities))
    # 归一化信息熵
    n_entropy = 1 - (entropy / np.log(probabilities.size))
    
    return n_entropy

def process_p2(row,i,p): #并行计算p=2时每种问题的情况
    start = time.time()
    CtrlSeries = row["formula"]
    Power_1 = {"Z_1":CtrlSeries[0], "Z_2":CtrlSeries[1], "Z_3":CtrlSeries[2],"Z_4":CtrlSeries[3]}
    Power_2 = {"Z_1 Z_2":CtrlSeries[4], "Z_3 Z_4":CtrlSeries[5],
            "Z_1 Z_3":CtrlSeries[6], "Z_1 Z_4":CtrlSeries[7], "Z_2 Z_3":CtrlSeries[8], "Z_2 Z_4":CtrlSeries[9]}
    Power_3 = {"Z_1 Z_2 Z_3":CtrlSeries[10], "Z_1 Z_2 Z_4":CtrlSeries[11], "Z_1 Z_3 Z_4": CtrlSeries[12] , "Z_2 Z_3 Z_4":CtrlSeries[13]}
    Power_4 = {"Z_1 Z_2 Z_3 Z_4":CtrlSeries[14]}
    Problem_beta = {"Power_1":Power_1,
            "Power_2":Power_2,
            "Power_3":Power_3,
            "Power_4":Power_4}
    Problem_alpha = create_new_dict(Problem_beta , modify_key_value)
    Problem = (Problem_alpha,Problem_beta, Constant, factor)
    #----------------- 获取CSP(Problem)
    model = CSP(Problem)
    HC = model.HC
    #loopSeries = list(itertools.product(gamma_1Lst,gamma_1Lst,beta_1Lst,beta_1Lst))
    E_lst = Parallel(n_jobs=1,verbose=0)(delayed(Qaoa3E)(Problem,HC,[gamma_1,gamma_2],[beta_1,beta_2],p,s) 
                    for gamma_1,gamma_2,beta_1,beta_2 in loopSeries)
    df = pd.DataFrame(loopSeries, columns=["gamma_1","gamma_2","beta_1","beta_2"]) #得到参数的数据框
    df.insert(0,column="E",value = E_lst) #将E_lst插入数据框
    row = df.loc[df["E"].idxmax(),:] # E_max对应列
    model.updateQC([row["gamma_1"],row["gamma_2"]],[row["beta_1"],row["beta_2"]],p)
    pureState = model.getState(s) # 返回该最优量子线路输出的纯态
    State = getBaseProb(pureState).real #得到概率向量
    idxMax = np.argmax(State) #binary变量为 二进制编码的解
    binary = bin(idxMax)[2:].zfill(4)
    #--------------- 上述为量子部分，下述为经典网格搜索部分
    ObjLst,Num_solution_Lst = GridSearch_CSP(Problem)
    New_Num_solution_Lst = []
    for lt in Num_solution_Lst:
        New_Num_solution_Lst.append(''.join(str(int((1-x)/2)) for x in lt))
    #------------输出期望，QAOA的最优解，概率向量的信息熵，gamma_1,gamma_2, beta_1,beta_2, 经典最优目标值向量(可能由多解)，经典最优解向量，QAOA的解是否为最优解之一
    end = time.time()
    print(i, " Time cost is minutes", np.round((end-start)/60,4))
    return row["E"],binary,Entropy(State),row["gamma_1"],row["gamma_2"],row["beta_1"],row["beta_2"],ObjLst,New_Num_solution_Lst,binary in New_Num_solution_Lst




In [101]:
"""
parser = argparse.ArgumentParser(description='CSP-问题8-p=2')
#type是要传入的参数的数据类型  help是该参数的提示信息
parser.add_argument('--Num', type=int,required=True, help='每个参数遍历的次数')
parser.add_argument('--jobs', type=int,required=True, help='核心数')
parser.add_argument('--External', type=int,required=True, help='外部项的个数')

args = parser.parse_args()
args = vars(args)
print(args)

"""
N = 4#args["Num"]#采样数
num_cores = 10 #args["jobs"]
External = 2#args["External"]
p = 2 # p = 1


combinations = itertools.product([0, -1, 1], repeat=15)
CtrlSeries_Lst = [c for c in combinations if sum(1 for i in c[6:] if i != 0) == External][:200]
df_CSP = pd.DataFrame(data = {"E":np.nan,
                            "solutions":np.nan,
                            "real_solutions":np.nan,
                            "isOpt":np.nan,
                            "Entropy":np.nan,
                            "gamma_1":np.nan,
                            "gamma_2":np.nan,
                            "beta_1":np.nan,
                            "beta_2":np.nan,
                            "formula":CtrlSeries_Lst})
Constant = 0
factor = 1

t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1) #创建初态
s = (_H() + _H() + _H() + _H()).dot(t).to_array()
gamma_1Lst = np.linspace(0,2*np.pi,N*2) #生成遍历参数列表
beta_1Lst = np.linspace(0,np.pi,N) 

loopSeries = list(itertools.product(gamma_1Lst,gamma_1Lst,beta_1Lst,beta_1Lst))
fill_data = Parallel(n_jobs=num_cores,timeout = None)(delayed(process_p2)(row,i,p) 
                    for i,row in tqdm(list(df_CSP.iterrows())))
fill_df = pd.DataFrame(data = fill_data,columns = ["E", "solutions", "Entropy","gamma_1","gamma_2", "beta_1", "beta_2", "MaxObj","real_solutions","isOpt"])

for col in fill_df: #将输出填入df_CSP数据框中
    df_CSP[col] = fill_df[col]
tmpLst = df_CSP.apply(lambda row: row["E"]/ row["MaxObj"][0],axis = 1)
df_CSP.insert(1,"ratio",value=tmpLst)
print("计算完成")
#df_CSP.round(8).to_csv("df_CSP_p1_External={}.csv".format(External),encoding = "utf-8")

In [4]:
Constant = 0
factor = 1
t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1) #创建初态
s = (_H() + _H() + _H() + _H()).dot(t).to_array()
#CtrlSeries = row["formula"]
Power_1 = {"Z_1": 1, "Z_2": 1, "Z_3": 0, "Z_4": -1}
Power_2 = {"Z_1 Z_2": 0, "Z_3 Z_4":0,
        "Z_1 Z_3": 0, "Z_1 Z_4": 0, "Z_2 Z_3": 0, "Z_2 Z_4": 0}
Power_3 = {"Z_1 Z_2 Z_3": -1, "Z_1 Z_2 Z_4":0, "Z_1 Z_3 Z_4": 0 , "Z_2 Z_3 Z_4":0}
Power_4 = {"Z_1 Z_2 Z_3 Z_4":1}
Problem_beta = {"Power_1":Power_1,
        "Power_2":Power_2,
        "Power_3":Power_3,
        "Power_4":Power_4}
Problem_alpha = create_new_dict(Problem_beta , modify_key_value)
Problem = (Problem_alpha,Problem_beta, Constant, factor)
#----------------- 获取CSP(Problem)
model = CSP(Problem)
HC = model.HC

In [5]:
model.updateQC([0,1],[0,1],2)
model.getExpctation(s)

1.0924388147242878

In [ ]:
N = 15

t = np.concatenate((np.array([1]) , np.zeros((15)))).reshape(-1,1) #创建初态
s = (_H() + _H() + _H() + _H()).dot(t).to_array()
gamma_1Lst = np.linspace(0,2*np.pi,N*2) #生成遍历参数列表
beta_1Lst = np.linspace(0,np.pi,N) 

loopSeries = list(itertools.product(gamma_1Lst,gamma_1Lst,beta_1Lst,beta_1Lst))
E_lst = Parallel(n_jobs=10,verbose=0)(delayed(Qaoa3E)(Problem,HC,[gamma_1,gamma_2],[beta_1,beta_2],p,s) 
                    for gamma_1,gamma_2,beta_1,beta_2 in tqdm(loopSeries))
df = pd.DataFrame(loopSeries, columns=["gamma_1","gamma_2","beta_1","beta_2"]) #得到参数的数据框
df.insert(0,column="E",value = E_lst) #将E_lst插入数据框
row = df.loc[df["E"].idxmax(),:] # E_max对应列
model.updateQC([row["gamma_1"],row["gamma_2"]],[row["beta_1"],row["beta_2"]],p)
pureState = model.getState(s) # 返回该最优量子线路输出的纯态
State = getBaseProb(pureState).real #得到概率向量
idxMax = np.argmax(State) #binary变量为 二进制编码的解
binary = bin(idxMax)[2:].zfill(4)
#--------------- 上述为量子部分，下述为经典网格搜索部分
ObjLst,Num_solution_Lst = GridSearch_CSP(Problem)
New_Num_solution_Lst = []
for lt in Num_solution_Lst:
    New_Num_solution_Lst.append(''.join(str(int((1-x)/2)) for x in lt))


100%|██████████| 202500/202500 [01:00<00:00, 3351.68it/s]


In [ ]:
df.sort_values(by = "E")

,E,gamma_1,gamma_2,beta_1,beta_2
181190,-2.567839,5.633201,5.416539,0.897598,1.121997
180965,-2.565775,5.633201,5.199877,0.897598,1.121997
181189,-2.541713,5.633201,5.416539,0.897598,0.897598
180964,-2.514296,5.633201,5.199877,0.897598,0.897598
180966,-2.503539,5.633201,5.199877,0.897598,1.346397
...,...,...,...,...,...
14463,2.953063,0.433323,0.866646,0.897598,0.673198
21003,2.968375,0.649985,0.649985,1.121997,0.673198
21004,2.970370,0.649985,0.649985,1.121997,0.897598
21214,3.057299,0.649985,0.866646,0.897598,0.897598


In [6]:
def getParaQC(Problem,gamma:float, beta:float):
    Problem_alpha = Problem[0]
    Problem_beta = Problem[1]
    QC = 1
    for (qubit_str, _), (_,coef) in zip(Problem_alpha["Power_1"].items(),Problem_beta["Power_1"].items()):
        if coef != 0:
            QC = _Rz(eval(qubit_str),4,coef*gamma) * QC
    for (qubit_str, _), (_,coef) in zip(Problem_alpha["Power_2"].items(),Problem_beta["Power_2"].items()):
        if coef != 0:
            qubit_idx = eval(qubit_str)
            QC = _Rzz(qubit_idx[0],qubit_idx[1],4,coef*gamma) * QC
    for (qubit_str, _), (_,coef) in zip(Problem_alpha["Power_3"].items(),Problem_beta["Power_3"].items()):
        if coef != 0:
            qubit_idx = eval(qubit_str)
            QC = _Rzzz(qubit_idx[0],qubit_idx[1],qubit_idx[2],4,coef*gamma) * QC

    if Problem_alpha["Power_4"]['0,1,2,3'] != 0:
        QC = _Rzzzz(Problem_alpha["Power_4"]['0,1,2,3']*gamma) *  QC
    QC = _Rx(0,4,beta)*_Rx(1,4,beta)*_Rx(2,4,beta)*_Rx(3,4,beta) * QC
    return QC 

def QC_func(Problem, reversed = False): #输出一个func，对应Problem
    model = CSP(Problem)
    HC = model.HC
    def func(x1,x2,x3,x4):
        #print(X)
        """if X.shape[0] == 4:
            gamma_1,beta_1,gamma_2,beta_2 = X[0],X[1],X[2],X[3]
        elif X.shape[0] ==2:
            gamma_1,beta_1,gamma_2,beta_2 = X[0],X[1],0,0
        else:
            raise ValueError("X 输入不对")
        """
        gamma_1,beta_1,gamma_2,beta_2 = x1,x2,x3,x4
        QC = getParaQC(Problem,gamma_2,beta_2) * getParaQC(Problem,gamma_1,beta_1)
        global s
        rb = QC.dot(s)
        E = (np.conjugate(rb).T.dot(HC).dot(rb))[0][0].real
        if reversed:
            #print(-E)
            return -E
        else:
            return E
    return func

In [7]:
func = QC_func(Problem)

In [8]:
func(1,0,2,4)

-0.21978990249050892

In [59]:
def black_box_function(x, y):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    return -x ** 2 - (y - 1) ** 2 + 1
from bayes_opt import BayesianOptimization

pbounds = {'x1': (0, 2*np.pi), 
           'x2': (0, 2*np.pi),
           'x3': (0, 2*np.pi),
           'x4': (0, 2*np.pi)}
optimizer = BayesianOptimization(
    f=func,
    pbounds=pbounds,
    random_state=1,
    
)

In [60]:
init_points = 20
n_iter = 200
x_lst = optimizer.maximize(
    init_points=init_points,
    n_iter=n_iter,
);

|   iter    |  target   |    x1     |    x2     |    x3     |    x4     |
-------------------------------------------------------------------------
| 1         | 0.06998   | 2.62      | 4.526     | 0.0007186 | 1.9       |
| 2         | 0.3527    | 0.9221    | 0.5802    | 1.17      | 2.171     |
| 3         | 0.6559    | 2.493     | 3.385     | 2.634     | 4.305     |
| 4         | -0.1995   | 1.285     | 5.517     | 0.1721    | 4.213     |
| 5         | 0.7271    | 2.622     | 3.51      | 0.8821    | 1.245     |
| 6         | -1.446    | 5.031     | 6.084     | 1.969     | 4.35      |
| 7         | 1.674     | 5.507     | 5.621     | 0.5343    | 0.2454    |
| 8         | 1.265     | 1.067     | 5.518     | 0.6179    | 2.646     |
| 9         | 0.5161    | 6.019     | 3.35      | 4.347     | 1.982     |
| 10        | 0.188     | 4.313     | 5.244     | 0.1149    | 4.713     |
| 11        | 0.2512    | 6.213     | 4.701     | 1.762     | 4.959     |
| 12        | -1.315    | 0.6486    | 

In [61]:
optimizer.max

{'target': 2.807527236766819,
 'params': {'x1': 5.631236933644537,
  'x2': 4.691547579511089,
  'x3': 0.6519528816460982,
  'x4': 0.7204715800235002}}

In [52]:
x_lst = x_lst[0:init_points] + [np.array(list(dt.values())) for dt in x_lst[init_points:]]

In [53]:
df_result = pd.DataFrame(x_lst,columns = ["gamma_1", "beta_1","gamma_2","beta_2"])
#[func(row["gamma_1"],row["beta_1"],row["gamma_2"],row["beta_2"]) for i,row in df_result.iterrows()]
E_lst = df_result.apply(lambda row: func(row["gamma_1"],row["beta_1"],row["gamma_2"],row["beta_2"]),axis = 1)
df_result.insert(0,"E",E_lst)

In [54]:
df_result

,E,gamma_1,beta_1,gamma_2,beta_2
0,0.069977,2.620227,4.525932,0.000719,1.899612
1,0.352717,0.922094,0.580181,1.170307,2.171222
2,0.655948,2.492964,3.385485,2.633877,4.305361
3,-0.199529,1.284611,5.517375,0.172081,4.212672
4,0.727068,2.622003,3.510352,0.882077,1.244708
5,-1.445907,5.031227,6.083767,1.969302,4.349991
6,1.674117,5.506515,5.620979,0.534349,0.245388
7,1.264633,1.067076,5.517532,0.617931,2.645897
8,0.516108,6.018597,3.349976,4.347192,1.982443
9,0.187981,4.313413,5.244108,0.114909,4.713296


In [55]:
optimizer.max

{'target': 2.111564518357395,
 'params': {'x1': 5.750798762540231,
  'x2': 5.386931047493921,
  'x3': 0.37454438314817196,
  'x4': 0.38394493475777447}}

In [ ]:
optimizer.

TypeError: Observable.subscribe() missing 2 required positional arguments: 'event' and 'subscriber'